In [2]:
%pip install pandas openpyxl transformers torch
%pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
max_context = 4096
llm = Llama(
    model_path,
    n_ctx=max_context
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/solar/OneDrive/Documents/Capstone/Capstone-Jupyter/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 12

In [4]:
import json
import re

with open("metadata.json", "r", encoding="utf-8") as f:
    raw_metadata = json.load(f)

# Create a lookup table
metadata_lookup = {
    entry["code"]: f'{entry["name"]}. {entry["description"]}' 
    for entry in raw_metadata
}

def extract_codes(query):
    return re.findall(r"\[([A-Za-z0-9_]+)\]", query)

list_keywords = {"IN", "ANY", "NONE"}
null_keywords = {":NC": "Null type \"Not collected\"", ":NA": "Null type \"Not applicable\"", ":ND": "Null type \"Not disclosed\"", ":NI": "Null type \"No information\"", ":NM": "Null type \"Not meaningful\""}
def extract_nulls_and_lists(query):
    nulls = []
    lists = []
    contains_null = re.findall(r"\[([A-Za-z0-9_]+)\]\s+IS\s+(:[A-Z]{2})", query)
    for code, null_keyword in contains_null:
        nulls.append((code, null_keyword))
    
    contains_list = re.findall(r"\[([A-Za-z0-9_]+)\]\s+(IN|ANY|NONE)\b", query, re.IGNORECASE)
    for code, list_keyword in contains_list:
        lists.append((code, list_keyword.upper()))
    
    return nulls, lists

# Find the entry with the longest name + description combo
longest_entry = max(metadata_lookup.items(), key=lambda item: len(item[1]))

# Print the result
print(f'Longest entry code: {longest_entry[0]}')
print(f'Length: {len(longest_entry[1])}')
print(f'Content: {longest_entry[1]}')

Longest entry code: EUTaxManIntSerElcRevOverAlign
Length: 1353
Content: EU Taxon - Man, Instal, Serv of Elec Equip Overall Align. EU Taxon - Man, Instal, Serv of Elec Equip Overall Align: This factor identifies the overall alignment for Manufacture, installation, and servicing of high, medium and low voltage electrical equipment for electrical transmission and distribution that result in or enable a substantial contribution to climate change mitigation, covering the substantial contribution criteria, the do no significant harm criteria, and the minimum social safeguards. This is the aggregated result across all Taxonomy objectives. The possible values are: Aligned,Aligned (>90%),Aligned (>80%),Aligned (>70%),Aligned (>60%),Aligned (>50%),Aligned (>40%),Aligned (>30%),Aligned (>20%),Aligned (>10%),Aligned (>0%),Likely aligned (100%),Likely aligned (>90%),Likely aligned (>80%),Likely aligned (>70%),Likely aligned (>60%),Likely aligned (>50%),Likely aligned (>40%),Likely aligned (>30%),Li

In [5]:
def max_tokens(text, max_tokens=40):
    #moved this to use in below function
    tokens = llm.tokenize(text.encode("utf-8"))
    if len(tokens) <= max_tokens:
        return text
    truncated = llm.detokenize(tokens[:max_tokens]).decode("utf-8", errors="ignore")
    return truncated + "..."

def build_ordered_context(query, token_budget):
    #builds context in-order: null type definitions, enumerations for list types, and metadata lookups
    context_lines = []
    seen = set()
    used_tokens = 0

    null_hits, list_hits = extract_nulls_and_lists(query)
    codes_in_order = re.findall(r"\[([A-Za-z0-9_]+)\]", query)

    #tokenizer = llm.tokenizer
    for code in codes_in_order:
        if code in seen:
            continue
        seen.add(code)

        null_entry = next((kw for c, kw in null_hits if c == code), None)
        if null_entry:
            null_def = null_keywords.get(null_entry, f"No definition for {null_entry}")
            line = f"{code} = {null_entry} → {null_def}"
            tokens = len(llm.tokenize(line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(line)
            used_tokens += tokens

        base_meta = metadata_lookup.get(code, "No metadata found.")
        metadata_line = f"{code}: {max_tokens(base_meta, 100)}" 
        tokens = len(llm.tokenize(metadata_line.encode("utf-8")))
        if used_tokens + tokens > token_budget:
            break
        context_lines.append(metadata_line)
        used_tokens += tokens

        if any(c == code for c, _ in list_hits):
            enum_line = f"{code} (enumeration): {max_tokens(base_meta, 100)}"
            tokens = len(llm.tokenize(enum_line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(enum_line)
            used_tokens += tokens

    return "\n".join(context_lines)

def max_afforded_tokens(codes):
    return max(4096 // max(1, len(codes)), 100)


def translate_query(query, max_total_tokens=2048, max_output_tokens=256):
    codes = extract_codes(query)

    m_tokens = max_afforded_tokens(codes)

    # Initial prompt pieces
    instruction = "### Instruction:\nRephrase the following ESGish query into a concise natural English sentence. Each query is asking for all companies or issuers that match some paramater. Use the following metadata definitions for clarity:\n\n"
    query_part = f"\n\nQuery: {query}\n\n### Response:"

    # Tokenize instruction and query to calculate remaining token budget
    #tokenizer = llm.tokenize  # Built-in tokenizer
    instruction_tokens = len(llm.tokenize(instruction.encode("utf-8")))
    query_tokens = len(llm.tokenize(query_part.encode("utf-8")))
    token_budget = max_total_tokens - max_output_tokens - instruction_tokens - query_tokens

    # Build full context blocks for each code
    context = build_ordered_context(query, token_budget)
    print("query: ", query)
    print(context)

    """
    # Now iteratively add context blocks until budget is exhausted
    context = ""
    used_tokens = 0
    for block in context_blocks:
        block_tokens = len(tokenizer(block.encode("utf-8"))) + 1  # +1 for newline
        if used_tokens + block_tokens <= token_budget:
            context += block + "\n"
            used_tokens += block_tokens
        else:
            break  # stop once we're out of budget
    """
    # Final prompt
    prompt = instruction + context + query_part

    # Call model
    response = llm(prompt, max_tokens=max_output_tokens, temperature=0.1)
    return response["choices"][0]["text"].strip()




In [6]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("NullType_10Queries.xlsx")
queries = df["Esgish"].tolist()

#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries_TEST.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")

query:  AND([ClawBackProvisionDisclosure] IS :ND,[CEOStockOwnershipGuideRatio] > '1')
ClawBackProvisionDisclosure = :ND → Null type "Not disclosed"
ClawBackProvisionDisclosure: Q155: Clawback Provision. Does the company have a clawback or malus provision? (Q155)
CEOStockOwnershipGuideRatio: Q145: CEO Stock Ownership Guidelines (% of Salary). What salary multiple of the CEO is subject to stock ownership guidelines? (Q145)


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    4566.29 ms /   191 tokens (   23.91 ms per token,    41.83 tokens per second)
llama_perf_context_print:        eval time =    4758.41 ms /    33 runs   (  144.19 ms per token,     6.94 tokens per second)
llama_perf_context_print:       total time =    9330.64 ms /   224 tokens
Llama.generate: 190 prefix-match hit, remaining 1 prompt tokens to eval


query:  AND([ClawBackProvisionDisclosure] IS :ND,[CEOStockOwnershipGuideRatio] > '1')
ClawBackProvisionDisclosure = :ND → Null type "Not disclosed"
ClawBackProvisionDisclosure: Q155: Clawback Provision. Does the company have a clawback or malus provision? (Q155)
CEOStockOwnershipGuideRatio: Q145: CEO Stock Ownership Guidelines (% of Salary). What salary multiple of the CEO is subject to stock ownership guidelines? (Q145)


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    4745.69 ms /    34 runs   (  139.58 ms per token,     7.16 tokens per second)
llama_perf_context_print:       total time =    4750.83 ms /    35 tokens
Llama.generate: 51 prefix-match hit, remaining 169 prompt tokens to eval


query:  AND([ClimateEmissionsFiscalYear] == '2022',[ClimateParentEntityID] IS :NA)
ClimateEmissionsFiscalYear: GHG Emissions - Fiscal Year. GHG Emissions - Fiscal Year: This factor provides the fiscal year associated with the corresponding GHG emissions values.
ClimateParentEntityID = :NA → Null type "Not applicable"
ClimateParentEntityID: Climate Parent Entity ID. Climate Parent Entity ID: This factor provides the identifier of the issuer‘s parent, from which the climate data is assigned. The climate data from the parent company is assigned to a subsidiary based on criteria such as industry classification, ownership structure, and business activities.


...

query:  OR(AND([PornographyNotAccessibeToMinors] IS :NA,[PornographyDistMinRev] IS :NA),AND([PornographyDistMinRev] <= '0.02',[PornographyNotAccessibeToMinors] False))
PornographyNotAccessibeToMinors = :NA → Null type "Not applicable"
PornographyNotAccessibeToMinors: Pornography - Not accessible to minors. Pornography - Not accessible to minors: This factor identifies pornographic materials which are specifically marked as not accessible to minors or are prohibited/restricted for those persons below 18 years of age.
PornographyDistMinRev = :NA → Null type "Not applicable"
PornographyDistMinRev: Pornography - Distribution Minimum Percentage of Revenues (%). Pornography - Distribution Minimum Percentage of Revenues (%): This factor identifies the minimum share of revenue estimated or reported to be derived from involvement in the distribution of pornography as a percentage of the issuers’ total revenue in the latest financial year.


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    5753.05 ms /   251 tokens (   22.92 ms per token,    43.63 tokens per second)
llama_perf_context_print:        eval time =   13218.44 ms /    95 runs   (  139.14 ms per token,     7.19 tokens per second)
llama_perf_context_print:       total time =   18989.85 ms /   346 tokens
Llama.generate: 50 prefix-match hit, remaining 273 prompt tokens to eval


query:  OR([TobaccoInvolvement] ANY 'Distribution|Services',[TobaccoInvolvement] IS :NC)
TobaccoInvolvement = :NC → Null type "Not collected"
TobaccoInvolvement: Tobacco - Involvement Tie. Tobacco - Involvement Tie: This factor identifies issuers engaged in the production, distribution, or provision of services related to tobacco. The "Production" value identifies issuers engaged in manufacturing and producing tobacco products, as well as companies that grow or process raw tobacco leaves, "Distribution" includes issuers engaged in the wholesale or retail distribution of tobacco products, and "Services"  identifies iss...
TobaccoInvolvement (enumeration): Tobacco - Involvement Tie. Tobacco - Involvement Tie: This factor identifies issuers engaged in the production, distribution, or provision of services related to tobacco. The "Production" value identifies issuers engaged in manufacturing and producing tobacco products, as well as companies that grow or process raw tobacco leaves, "Dist

llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    6292.50 ms /   273 tokens (   23.05 ms per token,    43.39 tokens per second)
llama_perf_context_print:        eval time =    5129.16 ms /    37 runs   (  138.63 ms per token,     7.21 tokens per second)
llama_perf_context_print:       total time =   11427.90 ms /   310 tokens


Saved batch 1 to translated_queries_TEST.xlsx


In [7]:
import pandas as pd

output_file = "translated_queries_TEST5.xlsx"

# Extract the English column
df = pd.read_excel(output_file)
english_queries = df["English"].tolist()

# Function to request a comprehensibility rating from the model
def rate_comprehensibility(text, max_tokens=256):
    # Adjust the prompt to ask the model for a rating from 1 to 10
    prompt = f"""### Instruction:
Please rate the following text's comprehensibility from 1 to 10, where:
- 1 = Completely incomprehensible, nonsensical, or full of errors.
- 5 = Understandable with effort; some awkwardness, complexity, or minor errors.
- 10 = Perfectly clear, natural, and easy to understand.

Here are some examples:
Text: "asjdk asjd aksd" → Rating: 1
Text: "Provide list companies ESG data incomplete understandable" → Rating: 4
Text: "Please provide a list of companies with complete ESG data." → Rating: 9

Now, rate this text:
Text: {text}

### Response:"""
    # Call model (adjust temperature and other params as needed)
    response = llm(prompt, max_tokens=max_tokens, temperature=0.2)
    rating = response["choices"][0]["text"].strip()
    print(rating)
    # Ensure the response is a valid number between 1 and 10
    try:
        rating = int(rating)
        if 1 <= rating <= 10:
            return rating
    except ValueError:
        pass
    return None  # Return None if no valid rating is obtained

# List to store ratings
ratings = []

# Iterate through each English translation and get a rating
for query in english_queries:
    rating = rate_comprehensibility(query)
    ratings.append(rating)

# Add the ratings as a new column to the dataframe
df["Comprehensibility Rating"] = ratings

# Save the updated dataframe with ratings to a new Excel file
df.to_excel(output_file, index=False)

print("Comprehensibility ratings added and saved to 'translated_queries_with_ratings.xlsx'")


Llama.generate: 6 prefix-match hit, remaining 206 prompt tokens to eval
llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    4678.38 ms /   206 tokens (   22.71 ms per token,    44.03 tokens per second)
llama_perf_context_print:        eval time =    9648.40 ms /    70 runs   (  137.83 ms per token,     7.26 tokens per second)
llama_perf_context_print:       total time =   14339.53 ms /   276 tokens
Llama.generate: 191 prefix-match hit, remaining 20 prompt tokens to eval


Rating: 6

The text is understandable but has some complexity and awkwardness. The use of technical terms such as "abortifacients" and "maximum share of revenue" may make it more challenging for some readers to fully comprehend. However, the overall structure and grammar of the text are clear.


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =     500.06 ms /    20 tokens (   25.00 ms per token,    39.99 tokens per second)
llama_perf_context_print:        eval time =    8782.47 ms /    64 runs   (  137.23 ms per token,     7.29 tokens per second)
llama_perf_context_print:       total time =    9293.47 ms /    84 tokens
Llama.generate: 166 prefix-match hit, remaining 44 prompt tokens to eval


Rating: 7

The text is understandable, but it requires some knowledge of financial terminology and the context of the topic. The sentence structure is complex, and there are some minor errors in the phrasing. However, overall, the text is clear and easy to understand with some effort.


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    1008.98 ms /    44 tokens (   22.93 ms per token,    43.61 tokens per second)
llama_perf_context_print:        eval time =    7579.05 ms /    55 runs   (  137.80 ms per token,     7.26 tokens per second)
llama_perf_context_print:       total time =    8597.58 ms /    99 tokens
Llama.generate: 166 prefix-match hit, remaining 21 prompt tokens to eval


...

Rating: 7

The text is mostly understandable, but it contains some complex and technical terms related to the query's criteria. The use of percentages and specific industry categories may require some effort to fully comprehend. However, the overall structure and grammar of the text is clear and easy to follow.


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =    1174.55 ms /    51 tokens (   23.03 ms per token,    43.42 tokens per second)
llama_perf_context_print:        eval time =    2488.85 ms /    18 runs   (  138.27 ms per token,     7.23 tokens per second)
llama_perf_context_print:       total time =    3666.48 ms /    69 tokens
Llama.generate: 164 prefix-match hit, remaining 26 prompt tokens to eval


The text is understandable with some effort. I would rate it a 6.


llama_perf_context_print:        load time =    4566.49 ms
llama_perf_context_print: prompt eval time =     607.33 ms /    26 tokens (   23.36 ms per token,    42.81 tokens per second)
llama_perf_context_print:        eval time =    9747.21 ms /    71 runs   (  137.28 ms per token,     7.28 tokens per second)
llama_perf_context_print:       total time =   10367.31 ms /    97 tokens


The rating for this text is 7. While there are no major errors, the text is somewhat complex and may require some effort to understand. The sentence structure is a bit awkward, and some words may be unfamiliar to those not familiar with the specific industry or terminology. However, overall, the text is relatively clear and easy to understand.
Comprehensibility ratings added and saved to 'translated_queries_with_ratings.xlsx'
